In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler, PowerTransformer,QuantileTransformer
from sklearn.model_selection import GridSearchCV, cross_val_score


In [26]:
base_path = os.path.join("/", "Volumes", "Extreme SSD", "ShellAi")
train_path = os.path.join(base_path, "train.csv")
test_path = os.path.join(base_path, "test.csv")

# Load the CSV files
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [50]:
# Creating  weighted columns
base_features = [col for col in train_df.columns if col not in [f"BlendProperty{i}" for i in range(1, 11)]]
targets = [f"BlendProperty{i}" for i in range(1, 11)]
weighted_data = {}
for i in range(1, 6):
    for j in range(1, 11):
        prop_col = f"Component{i}_Property{j}"
        frac_col = f"Component{i}_fraction"
        weighted_col = f"Weighted_Component{i}_Property{j}"
        weighted_data[weighted_col] = train_df[prop_col] * train_df[frac_col]

# Combine
data_with_weighted = pd.concat([train_df[base_features], pd.DataFrame(weighted_data)], axis=1)
data_with_weighted = data_with_weighted.copy()  # De-fragmented copy

In [30]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data_with_weighted)
scaled_df_features = pd.DataFrame(scaled_features, columns=data_with_weighted.columns)


pt = PowerTransformer(method='yeo-johnson')
transformed_targets_df = pd.DataFrame(pt.fit_transform(train_df[targets]), columns=targets, index=train_df.index)

# Initial feature selection using Random Forest feature importance
top_features = {}
for target in targets:
    X = scaled_df_features
    y = transformed_targets_df[target]
    rf_initial = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_initial.fit(X, y)
    feature_importance = pd.Series(rf_initial.feature_importances_, index=scaled_df_features.columns).sort_values(ascending=False)
    top_features[target] = feature_importance.head(20).index.tolist()

# GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20,]

}


In [34]:
# Train and evaluate
for target in targets:
    X = scaled_df_features[top_features[target]]
    y = transformed_targets_df[target]


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # GridSearchCV for hyperparameter tuning
    rf = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_absolute_percentage_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Best model
    best_rf = grid_search.best_estimator_
    print(f"Best params for {target}: {grid_search.best_params_}")


    cv_mape = -cross_val_score(best_rf, X_train, y_train, cv=5, scoring='neg_mean_absolute_percentage_error').mean() * 100
    print(f"Cross-validated MAPE for {target} (transformed): {cv_mape:.2f}%")


    y_pred = best_rf.predict(X_test)

    y_test_full = pd.DataFrame(np.column_stack([y_test] + [np.zeros(len(y_test))]*(len(targets)-1)), columns=targets, index=X_test.index)
    y_pred_full = pd.DataFrame(np.column_stack([y_pred] + [np.zeros(len(y_pred))]*(len(targets)-1)), columns=targets, index=X_test.index)
    y_test_inv = pt.inverse_transform(y_test_full)[targets.index(target)]
    y_pred_inv = pt.inverse_transform(y_pred_full)[targets.index(target)]

    # Handle zero/near-zero values in MAPE calculation
    mask = y_test_inv != 0
    test_mape = np.mean(np.abs((y_test_inv[mask] - y_pred_inv[mask]) / y_test_inv[mask]) * 100) if mask.any() else 0.0
    print(f"Test MAPE for {target} (original scale): {test_mape:.2f}%")


Best params for BlendProperty1: {'max_depth': 20, 'n_estimators': 100}
Cross-validated MAPE for BlendProperty1 (transformed): 130.41%
Test MAPE for BlendProperty1 (original scale): 5.78%
Best params for BlendProperty2: {'max_depth': 10, 'n_estimators': 100}
Cross-validated MAPE for BlendProperty2 (transformed): 105.18%
Test MAPE for BlendProperty2 (original scale): 1.88%
Best params for BlendProperty3: {'max_depth': 20, 'n_estimators': 200}
Cross-validated MAPE for BlendProperty3 (transformed): 113.10%
Test MAPE for BlendProperty3 (original scale): 1.46%
Best params for BlendProperty4: {'max_depth': 20, 'n_estimators': 100}
Cross-validated MAPE for BlendProperty4 (transformed): 114.85%
Test MAPE for BlendProperty4 (original scale): 47.21%
Best params for BlendProperty5: {'max_depth': 10, 'n_estimators': 100}
Cross-validated MAPE for BlendProperty5 (transformed): 4.50%
Test MAPE for BlendProperty5 (original scale): 0.01%
Best params for BlendProperty6: {'max_depth': 20, 'n_estimators': 

In [41]:
qt = QuantileTransformer(output_distribution='normal', random_state=42)
transformed_targets_df = pd.DataFrame(qt.fit_transform(train_df[targets]),
                                      columns=targets, index=train_df.index)
top_features = {}
for target in targets:
    X = scaled_df_features
    y = transformed_targets_df[target]
    rf_initial = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_initial.fit(X, y)
    feature_importance = pd.Series(rf_initial.feature_importances_, index=X.columns).sort_values(ascending=False)
    top_features[target] = feature_importance.head(20).index.tolist()


param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20]
}

print("\n📊 RANDOM FOREST RESULTS WITH QUANTILE TRANSFORMER\n" + "="*50)

for target in targets:
    print(f"\n🔹 Target: {target}")

    X = scaled_df_features[top_features[target]]
    y = transformed_targets_df[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    rf = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_absolute_percentage_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_rf = grid_search.best_estimator_
    print(f"Best params: {grid_search.best_params_}")

    # CV MAPE on transformed data
    cv_mape = -cross_val_score(best_rf, X_train, y_train, cv=5, scoring='neg_mean_absolute_percentage_error').mean() * 100
    print(f"Cross-validated MAPE (transformed): {cv_mape:.2f}%")

    # Predict on test set
    y_pred = best_rf.predict(X_test)

    # Prepare for inverse transform
    y_test_full = pd.DataFrame(np.column_stack([y_test] + [np.zeros(len(y_test))]*(len(targets)-1)),
                               columns=targets, index=X_test.index)
    y_pred_full = pd.DataFrame(np.column_stack([y_pred] + [np.zeros(len(y_pred))]*(len(targets)-1)),
                               columns=targets, index=X_test.index)

    y_test_inv = qt.inverse_transform(y_test_full)[targets.index(target)]
    y_pred_inv = qt.inverse_transform(y_pred_full)[targets.index(target)]

    # Calculate safe MAPE
    mask = y_test_inv != 0
    test_mape = np.mean(np.abs((y_test_inv[mask] - y_pred_inv[mask]) / y_test_inv[mask]) * 100) if mask.any() else 0.0
    print(f"Test MAPE (original scale): {test_mape:.2f}%")


📊 RANDOM FOREST RESULTS WITH QUANTILE TRANSFORMER

🔹 Target: BlendProperty1
Best params: {'max_depth': 20, 'n_estimators': 100}
Cross-validated MAPE (transformed): 129.01%
Test MAPE (original scale): 5.96%

🔹 Target: BlendProperty2
Best params: {'max_depth': 20, 'n_estimators': 200}
Cross-validated MAPE (transformed): 135.85%
Test MAPE (original scale): 2.72%

🔹 Target: BlendProperty3
Best params: {'max_depth': 10, 'n_estimators': 200}
Cross-validated MAPE (transformed): 233.63%
Test MAPE (original scale): 1.93%

🔹 Target: BlendProperty4
Best params: {'max_depth': 20, 'n_estimators': 200}
Cross-validated MAPE (transformed): 153.62%
Test MAPE (original scale): 77.99%

🔹 Target: BlendProperty5
Best params: {'max_depth': 20, 'n_estimators': 200}
Cross-validated MAPE (transformed): 3.55%
Test MAPE (original scale): 0.00%

🔹 Target: BlendProperty6
Best params: {'max_depth': 20, 'n_estimators': 200}
Cross-validated MAPE (transformed): 166.24%
Test MAPE (original scale): 2.03%

🔹 Target: Ble